<a href="https://colab.research.google.com/github/CC-Shivansh-Gupta/logistic-_reg/blob/main/logreg_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import matplotlib as plt
import copy, math

data = pd.read_csv('framingham.csv')
data.columns = data.columns.str.replace(' ', '_')
data
#print(type(std_data))
#print(std_data.info())

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1,50,1.0,1,1.0,0.0,0,1,0,313.0,179.0,92.0,25.97,66.0,86.0,1
4234,1,51,3.0,1,43.0,0.0,0,0,0,207.0,126.5,80.0,19.71,65.0,68.0,0
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0


In [18]:
data.fillna(int("0") , inplace = True)
data
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [19]:
def normalise_data(arr):
  #min_val = np.min(arr)
  #max_val = np.max(arr)
  mean = np.mean(arr)
  std = np.std(arr)
  norm_arr = np.zeros(len(arr))
  for i in range(len(arr)):
    norm_arr[i] = (arr[i] - mean) / std
  return norm_arr

In [20]:
for i in range(data.shape[1]-1):
  data.iloc[:, i] = normalise_data(data.iloc[:, i])
data

<ipython-input-20-6893fa0e634f>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.1531919  -0.86715836  1.1531919  ... -0.86715836 -0.86715836
 -0.86715836]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, i] = normalise_data(data.iloc[:, i])
<ipython-input-20-6893fa0e634f>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-1.23495068 -0.41825733 -0.18491638 ... -0.18491638 -0.65159829
  0.28176554]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, i] = normalise_data(data.iloc[:, i])
<ipython-input-20-6893fa0e634f>:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.98827076 -0.98827076  1.01186844 ...  1.01186844  1.01186844
 -0.98827076]' has dtype incompatible 

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1.153192,-1.234951,1.966086,-0.988271,-0.751322,-0.173612,-0.077033,-0.671101,-0.162477,-0.760870,-1.195907,-1.082625,0.290396,0.342628,0.077208,0
1,-0.867158,-0.418257,0.066560,-0.988271,-0.751322,-0.173612,-0.077033,-0.671101,-0.162477,0.314116,-0.515187,-0.158988,0.688549,1.584339,0.046781,0
2,1.153192,-0.184916,-0.883204,1.011868,0.929210,-0.173612,-0.077033,-0.671101,-0.162477,0.216390,-0.220209,-0.242955,-0.078348,-0.071276,-0.135778,0
3,-0.867158,1.331800,1.016323,1.011868,1.769476,-0.173612,-0.077033,1.490089,-0.162477,-0.174514,0.800871,1.016549,0.654615,-0.899083,0.868299,1
4,-0.867158,-0.418257,1.016323,1.011868,1.181289,-0.173612,-0.077033,-0.671101,-0.162477,0.998198,-0.106755,0.092912,-0.585088,0.756531,0.320621,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1.153192,0.048425,-0.883204,1.011868,-0.667296,-0.173612,-0.077033,1.490089,-0.162477,1.545464,2.116930,0.764648,0.064173,-0.816302,0.351047,1
4234,1.153192,0.165095,1.016323,1.011868,2.861821,-0.173612,-0.077033,-0.671101,-0.162477,-0.526328,-0.265590,-0.242955,-1.351985,-0.899083,-0.196631,0
4235,-0.867158,-0.184916,0.066560,1.011868,0.929210,-0.173612,-0.077033,-0.671101,-0.162477,0.275026,-0.061374,-0.914691,-0.833933,0.673751,0.351047,0
4236,-0.867158,-0.651598,-0.883204,1.011868,0.509077,-0.173612,-0.077033,-0.671101,-0.162477,-0.467692,-0.265590,0.344813,-1.476407,0.839312,-2.265639,0


In [21]:
number = data.shape[0]
print(number)
number_train = int(number * 9 / 10)
print(number_train)
lam = 0.1
alp = 1

4238
3814


In [22]:
x_train_list = []
for j in range(data.shape[1]):
    xj_train = np.zeros(number_train)
    for i in range(number_train):
        xj_train[i] = data.iloc[i, j]
    x_train_list.append(xj_train)  # Append xj_train directly to the list

# Separate the features (x_train) from the target (y_train)
x_train_tup = tuple(x_train_list[:-1]) # Exclude the last column (target)
x_train = np.column_stack(x_train_tup)
y_train = x_train_list[-1]  # The last column is the target

print(x_train)
print(y_train)

[[ 1.1531919  -1.23495068  1.96608614 ...  0.29039572  0.34262779
   0.07720798]
 [-0.86715836 -0.41825733  0.06655962 ...  0.68854851  1.58433854
   0.0467814 ]
 [ 1.1531919  -0.18491638 -0.88320363 ... -0.07834806 -0.0712758
  -0.13577807]
 ...
 [ 1.1531919  -1.7016326   0.06655962 ... -1.28864206  0.34262779
  -0.22705781]
 [-0.86715836  1.33179984 -0.88320363 ... -0.56246566 -0.89908297
  -0.28791097]
 [-0.86715836 -0.65159829  1.01632288 ...  0.4238674  -0.0712758
   0.0467814 ]]
[0. 0. 0. ... 0. 0. 0.]


In [23]:
w_para = np.zeros(data.shape[1] - 1)
print(w_para)
b_para = 0
print(b_para)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0


In [24]:
def prediction(x, w, b):
  c = np.dot(x, w) + b
  d = 1 / (1 + np.exp(-c))
  return d

In [25]:
def predict():
  predic= np.zeros(number_train)
  for i in range(number_train):
    predic[i] = prediction(x_train[i], w_para, b_para)
    #print(predic)
  return predic


In [26]:
pred = predict()

In [27]:
def loss_calc(pred, y):
  loss = -((y * math.log(pred)) + ((1 - y) * math.log(1 - pred)))
  return loss

In [28]:
def lossss():
  losses = np.zeros(number_train)
  for i in range(number_train):
    losses[i] = loss_calc(pred[i], y_train[i])
  #print(losses)
  return losses

In [29]:
loss = lossss()

In [30]:
def cost_function(loss, l, w):
  costie = np.mean(loss) + (l * np.sum(np.square(w)))/(2*len(loss))
  #print(costie)
  return costie

In [31]:
cost = cost_function(loss, lam, w_para)

In [32]:
def grad_desc(w, b, cost, pred, x, y, l, a, n):
  temp_w = copy.deepcopy(w)
  dj_dw = np.zeros(len(w))
  for k in range(len(w)):
    for i in range(number_train):
      dj_dw[k] = (((pred[i] - y[i])/n)*x[i][k]) + ((l*w[k])/n)
      temp_w[k] = w[k] - (a * dj_dw[k])
      dj_db = (pred[i] - y[i])/n
      temp_b = b - (a * dj_db)
    w[k] = temp_w[k]
    b = temp_b
  return w, b
  #print(w_para)
  #print(b_para)

In [33]:
w_para, b_para = grad_desc(w_para, b_para, cost, pred, x_train, y_train, lam, alp, number_train)

In [34]:
i = 0
while cost > 0.1:
  pred = predict()
  loss = lossss()
  cost = cost_function(loss, lam, w_para)
  w_para, b_para = grad_desc(w_para, b_para, cost, pred, x_train, y_train, lam, alp, number_train)
  print(f'{i}th iteration, {cost}')
  i += 1
else:
  print('done')
  print(w_para)
  print(b_para)
  print(cost)
  print(f'done in {i} iterations')


0th iteration, 0.6924442929025
1th iteration, 0.6917433368743596
2th iteration, 0.6910443072284764
3th iteration, 0.6903471987265328
4th iteration, 0.6896520061389866
5th iteration, 0.6889587242451166
6th iteration, 0.688267347833065
7th iteration, 0.6875778716998812
8th iteration, 0.6868902906515636
9th iteration, 0.686204599503103
10th iteration, 0.6855207930785226
11th iteration, 0.6848388662109192
12th iteration, 0.6841588137425026
13th iteration, 0.6834806305246368
14th iteration, 0.6828043114178765
15th iteration, 0.6821298512920074
16th iteration, 0.6814572450260828
17th iteration, 0.6807864875084613
18th iteration, 0.6801175736368423
19th iteration, 0.6794504983183031
20th iteration, 0.6787852564693337
21th iteration, 0.6781218430158712
22th iteration, 0.6774602528933346
23th iteration, 0.6768004810466584
24th iteration, 0.6761425224303249
25th iteration, 0.6754863720083976
26th iteration, 0.6748320247545525
27th iteration, 0.6741794756521098
28th iteration, 0.6735287196940646


KeyboardInterrupt: 

In [35]:
print(w_para)
print(b_para)

[ 0.09476398  0.07120735 -0.11106483  0.10799927  0.08210529  0.01897246
  0.0084182   0.07333862  0.01775561 -0.04927838  0.01662565 -0.09273757
 -0.04632068  0.00778758 -0.00511132]
-1.697721021603239
